# Step 3. Predicting
For Predicting year,make,model of new cars' images, put images into **cars_predict** folder and boundation annotations in **devkit** folder with name *cars_predict.mat*.  
**If cropping of images(to be predicted) required depending on bounding boxes,give True in variable IsCropRequired else give False**

In [0]:
IsCropRequired = False

In [16]:
!pip install console_progressbar

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *
import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.io
import os
import shutil
from console_progressbar import ProgressBar

In [0]:
path='/content'

In [0]:
#from google.colab import drive
#drive.mount(path+'/gdrive')

In [0]:
def ensure_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

def save_test_data(fnames, bboxes):
    src_folder = path+'/cars_predict'
    dst_folder = path+'/predict'
    num_samples = len(fnames)

    pb = ProgressBar(total=100, prefix='Save predict data', suffix='', decimals=3, length=50, fill='=')

    for i in range(num_samples):
        fname = fnames[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        # print(fname)
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        dst_path = os.path.join(dst_folder, fname)
        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)

def process_predict_data():
    print("Processing predict data...")
    cars_annos = scipy.io.loadmat(path+'/devkit/cars_predict_annos')
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    bboxes = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        fname = annotation[0][4][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        fnames.append(fname)

    save_test_data(fnames, bboxes)


Pre-processing the prediction dataset so that existing model can be used for prediction.

In [0]:
if IsCropRequired:
    img_width, img_height = 224, 224
    ensure_folder(path+'/predict')

    process_predict_data()
    # clean up
    shutil.rmtree(path+'/cars_predict')
    # shutil.rmtree('devkit')
else:
  os.rename(path+'/cars_predict',path+'/predict')

In [0]:
cars_meta = sio.loadmat(path+'/devkit/cars_meta')
class_names = list(cars_meta['class_names'][0])  # shape=(1, 196)
class_names = [i[0] for i in class_names]

Loading trained model(ResNet152) which was trained in Step 2(Training and validation notebook)

In [22]:
data2 = ImageDataBunch.single_from_classes(path+'/predict/',class_names,ds_tfms=get_transforms(),size=224).normalize(imagenet_stats)
mdl = cnn_learner(data2, models.resnet152)
mdl.load(path+'/models/resnet152')

Learner(data=ImageDataBunch;

Train: LabelList (0 items)
x: ImageList

y: CategoryList

Path: /content/predict;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: /content/predict;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1

In [0]:
path1=path+'/predict'
imgs=[f for f in os.listdir(path1) if os.path.isfile(os.path.join(path1, f))]
result = pd.DataFrame()
for i in imgs:
    img=open_image(path+'/predict/'+i)
    car,idx,output = mdl.predict(img)
    prob=max(np.array(output))
    #print(car,prob,idx)
    result = result.append({'Image' : i , 'PredictedClass' : car, 'Probability':prob, 'ClassIndex':int(idx)} , ignore_index=True)
result.to_csv(path+'/PredictedClasses.csv')